In [1]:
%load_ext autoreload

In [2]:
%autoreload now

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
os.environ["WANDB_API_KEY"] = ""

# Download and convert data

In [4]:
from document_segmentation.pagexml.annotations.renate_analysis import (
    RenateAnalysis,
    RenateAnalysisInv,
)
from document_segmentation.pagexml.annotations.sheet import Sheet
from document_segmentation.settings import RENATE_ANALYSIS_SHEETS

sheets: list[Sheet] = [RenateAnalysis()] + [
    RenateAnalysisInv(sheet) for sheet in RENATE_ANALYSIS_SHEETS
]

In [6]:
N = 5

inventories = {
    sheet: list(sheet.all_annotated_inventories(N), shuffle=True) for sheet in sheets
}
inventories

Loading Inventories:   0%|          | 0/5 [00:00<?, ?inventory/s]

Loading Inventories:  20%|██        | 1/5 [00:01<00:04,  1.16s/inventory]


{<document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysis at 0x1695d0810>: [Inventory(inv_nr=1060, inventory_part=, pages=14 pages),
  Inventory(inv_nr=1066, inventory_part=, pages=1 pages),
  Inventory(inv_nr=1066, inventory_part=, pages=2 pages),
  Inventory(inv_nr=1070, inventory_part=, pages=3 pages),
  Inventory(inv_nr=1072, inventory_part=, pages=7 pages),
  Inventory(inv_nr=1072, inventory_part=, pages=12 pages),
  Inventory(inv_nr=1072, inventory_part=, pages=3 pages),
  Inventory(inv_nr=1074, inventory_part=, pages=16 pages)],
 <document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysisInv at 0x14e355710>: [Inventory(inv_nr=1547, inventory_part=, pages=688 pages)],
 <document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysisInv at 0x16861a590>: [Inventory(inv_nr=1557, inventory_part=, pages=1662 pages)]}

# Load Data

In [9]:
TRAINING_DATA = 0.8

In [7]:
import random

for sheet, sheet_inventories in inventories.items():
    random.shuffle(sheet_inventories)
inventories

{<document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysis at 0x1695d0810>: [Inventory(inv_nr=1066, inventory_part=, pages=1 pages),
  Inventory(inv_nr=1074, inventory_part=, pages=16 pages),
  Inventory(inv_nr=1072, inventory_part=, pages=7 pages),
  Inventory(inv_nr=1072, inventory_part=, pages=3 pages),
  Inventory(inv_nr=1066, inventory_part=, pages=2 pages),
  Inventory(inv_nr=1060, inventory_part=, pages=14 pages),
  Inventory(inv_nr=1070, inventory_part=, pages=3 pages),
  Inventory(inv_nr=1072, inventory_part=, pages=12 pages)],
 <document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysisInv at 0x14e355710>: [Inventory(inv_nr=1547, inventory_part=, pages=688 pages)],
 <document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysisInv at 0x16861a590>: [Inventory(inv_nr=1557, inventory_part=, pages=1662 pages)]}

In [10]:
# the first 80% of the RenateAnalysis sheet + the first of the RenateAnalysisInv sheets
_renate_analysis_inventories = list(inventories.values())[0]
split = int(TRAINING_DATA * len(_renate_analysis_inventories))
_renate_analysis_per_inv_1 = list(inventories.values())[1]

training_data = _renate_analysis_inventories[:split] + _renate_analysis_per_inv_1
training_data

[Inventory(inv_nr=1066, inventory_part=, pages=1 pages),
 Inventory(inv_nr=1074, inventory_part=, pages=16 pages),
 Inventory(inv_nr=1072, inventory_part=, pages=7 pages),
 Inventory(inv_nr=1072, inventory_part=, pages=3 pages),
 Inventory(inv_nr=1066, inventory_part=, pages=2 pages),
 Inventory(inv_nr=1060, inventory_part=, pages=14 pages),
 Inventory(inv_nr=1547, inventory_part=, pages=688 pages)]

In [11]:
validation_data = _renate_analysis_inventories[split:] + list(inventories.values())[2]
validation_data

[Inventory(inv_nr=1070, inventory_part=, pages=3 pages),
 Inventory(inv_nr=1072, inventory_part=, pages=12 pages),
 Inventory(inv_nr=1557, inventory_part=, pages=1662 pages)]

# Train Model

In [12]:
EPOCHS = 3

In [13]:
%autoreload now

In [14]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [15]:
tagger._device

'mps'

In [16]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
      )
      (_region_type): Embedding(10, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_featur

In [17]:
tagger.train_(training_data, validation_data, epochs=EPOCHS, shuffle=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: carschno. Use `wandb login --relogin` to force relogin


Evaluating: 100%|██████████| 3/3 [01:06<00:00, 22.15s/batch]
        [1],
        [3],
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 3/3 [00:01<00:00,  2.59batch/s]
        [3],
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 3/3 [00:01<00:00,  2.68batch/s]
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


MulticlassAccuracy,▁██
epoch,▁▅█
inventory length,▁▁█▁▁▁█▁▁▁▁▁▁▁▁█▁▁
loss,▂▂█▂▂▂█▂▂▂▂▁▁▁▂█▁▁
MulticlassAccuracy,0.78235
epoch,2
inventory length,16
loss,6.0985


# Evaluate Model

In [18]:
import sys

precision, recall, f1, accuracy = tagger.eval_(validation_data, sys.stdout)

Predicted	Actual	Page ID	Text	Scores


Evaluating:   0%|          | 0/3 [00:00<?, ?batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_1070_0199.jpg	2 saeckers Elck van 3000 lb; 2 halve dittos elck v	[0.004127567633986473, 0.5704946517944336, 0.3082047700881958, 0.11204306781291962, 0.0051299212500452995]
IN	IN	NL-HaNA_1.04.02_1070_0200.jpg		[0.004079905338585377, 0.06370437890291214, 0.6561031341552734, 0.2712872624397278, 0.0048252884298563]
END	END	NL-HaNA_1.04.02_1070_0201.jpg	Adriaen gerritsz van utrecht sergiant; marijn Ding	[0.004105671774595976, 0.017192602157592773, 0.2829276919364929, 0.6917695999145508, 0.004004433751106262]
BEGIN	BEGIN	NL-HaNA_1.04.02_1072_0487.jpg	201; Donderdach adij 19 novembr @:o 1620 stilo nou	[0.025836799293756485, 0.6167110800743103, 0.2751430571079254, 0.05036561191082001, 0.03194344788789749]
BEGIN	IN	NL-HaNA_1.04.02_1072_0488.jpg	opcoopp. in gevat en.; sesctup crmysorenl ap pedra	[0.01999642886221409, 0.4716133773326874, 0.41480404138565063, 0.0690336674451828, 0.02455255202949047]
IN	IN	NL-HaNA_1.04.02_1072_0489.jpg	Donderdach adij xvij decbr @ o 1620

Evaluating: 100%|██████████| 3/3 [00:01<00:00,  1.61batch/s]

IN	IN	NL-HaNA_1.04.02_1557_1348.jpg	Van Bengale onder dato 18:' Januarij A=o 1694.; ge	[0.014294532127678394, 0.1017485037446022, 0.7584106922149658, 0.1090242862701416, 0.016521988436579704]
IN	IN	NL-HaNA_1.04.02_1557_1349.jpg	Van Bengale onder dato 18=en Januarij A:o1694.; he	[0.014749487861990929, 0.10082494467496872, 0.7637374997138977, 0.10376229882240295, 0.01692572981119156]
IN	IN	NL-HaNA_1.04.02_1557_1350.jpg	Van Bengale onder dato 18=en Januarij A=o 1694.; P	[0.014314578846096992, 0.10928602516651154, 0.7621894478797913, 0.09774842113256454, 0.016461513936519623]
IN	IN	NL-HaNA_1.04.02_1557_1351.jpg	Van Bengale onder dato 18:en Januarij A:o 1694.; '	[0.013325954787433147, 0.11639368534088135, 0.7596113681793213, 0.095270074903965, 0.015398912131786346]
IN	IN	NL-HaNA_1.04.02_1557_1352.jpg	Van Bengale onder dato 18=en Januarij A:o 1694.; —	[0.012161015532910824, 0.11585263907909393, 0.7599310278892517, 0.09795379638671875, 0.014101531356573105]
IN	IN	NL-HaNA_1.04.02_1557_1353.jpg

In [19]:
import csv
import sys

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout, fieldnames=["Metric"] + [label.name for label in Label], delimiter="\t"
)

writer.writeheader()

for metric in precision, recall, f1:
    assert metric.average is None

    scores: list[float] = metric.compute().tolist()
    writer.writerow(
        {"Metric": metric.__class__.__name__}
        | {label.name: f"{score:.4f}" for label, score in zip(Label, scores)}
    )

assert accuracy.average is not None

print(
    f"{accuracy.__class__.__name__} ({accuracy.average} average):\t{accuracy.compute().item():.4f}",
    file=sys.stdout,
)

        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.


Metric	UNK	BEGIN	IN	END	OUT
MulticlassPrecision	0.0000	0.6667	0.7824	1.0000	0.0000
MulticlassRecall	0.0000	0.0253	0.9992	0.0056	0.0000
MulticlassF1Score	0.0000	0.0488	0.8776	0.0112	0.0000
MulticlassAccuracy (micro average):	0.7823
